In [3]:
%load_ext autoreload
%autoreload 2
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Flatten, LSTM
import pandas as pd
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from RainPredictionMachine.data import CleanDataRpm
clean_data = CleanDataRpm()
df = clean_data.clean_data(1)

In [12]:
from RainPredictionMachine.Trainer import pipe_creator
from sklearn import set_config
full_pipe, X_train, y_train  = pipe_creator(df)
set_config(display="diagram")
full_pipe

/Users/isabelagaio/.pyenv/versions/3.8.12/envs/rain-prediction-machine/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


TypeError: __init__() takes 2 positional arguments but 4 were given

In [ ]:
X_train_t.shape

In [ ]:
X_test_t.shape

In [ ]:
X_train_t1 = np.expand_dims(X_train_t, 2)

In [ ]:
X_train_t1.shape

In [ ]:
X_test_t1 = np.expand_dims(X_test_t, 2)

In [ ]:
X_test_t1.shape

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_pad = pad_sequences(X_train_t1)
X_pad = pad_sequences(X_train_t1, dtype= 'float32', padding= 'post', value=-1) 
X_pad.sort()

In [ ]:
normalizer = Normalization()
normalizer.adapt(X_train_t)
normalizer(X_train_t)[0]

In [ ]:
norm = Normalization()

model = Sequential()

model.add(norm)

model.add(LSTM(units=20, activation='tanh'))
model.add(Dense(10, activation="tanh"))
model.add(Dense(4, activation="softmax"))

model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accurancy'])

model.fit(X_test_t1, y_train)


In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(patience=5, restore_best_weights=True)

model.fit(X_train_t1, y_train, batch_size=16, epochs=1000, verbose=1,
         validation_split=0.2,
          callbacks=[es])